In [ ]:
import numpy as np
import math
from datetime import datetime
import random

SNR = { 
    1:	-9.478,
    2:	-6.658,
    3:	-4.098,
    4:	-1.798,
    5:	0.399,
    6:	2.424,
    7:	4.489,
    8:	6.367,
    9:	8.456,
    10:	10.266,
    11:	12.218,
    12:	14.122,
    13:	15.849,
    14:	17.786,
    15:	19.809,
    16:	0
}

SpectralEfficiency = {
    1:	0.15237,
    2:	0.2344,
    3:	0.377,
    4:	0.6016,
    5:	0.877,
    6:	1.1758,
    7:	1.4766,
    8:	1.9141,
    9:	2.4063,
    10:	2.7305,
    11:	3.3223,
    12:	3.9023,
    13:	4.5234,
    14:	5.1152,
    15:	5.5547
}

trafficType = {
    'ftp3': {'packetSize': 2, 'intensity': 1/30}
}

TxArray = "8x8"
RxArray = "2x2"
AntennaArrayGain = {
    "256x256":	41.59,
    "128x128":	35.58,
    "64x64":	29.58,
    "32x32":	23.58,
    "16x16":	17.58,
    "8x8":	11.57,
    "4x4":	5.57,
    "2x2":	0.43
}

bandwidth = 400 # MHz
InterferenceMargin = 3
Noise = -173.93
pathLoss = 2.1
fOp = 28 # in GHz
TxPower = 2 # in Watt
TxGain = AntennaArrayGain[TxArray]
RxGain = AntennaArrayGain[RxArray]
AMult = pow(10, 2 * math.log10(fOp)+3.24)

BSHeight = 10
UEHeight = 1.5

packetSize = 0.5 #Mbit
RBSizeMHz = 0.2
RBNum = bandwidth / RBSizeMHz

minMCS = 10

In [15]:
def get_communication_distance(mcs):
    if mcs > len(SNR)-1: return 0
    else: return math.sqrt(((TxPower * 10 ** 3 * 10 ** (TxGain / 10) * 10  ** (RxGain / 10)) / 
                      (AMult * (10 ** (Noise / 10) * bandwidth * 10 ** 6 + (10 ** (InterferenceMargin / 10))) / 
                       10 ** 3 * 10 ** (SNR[mcs] / 10)) ** (1 / pathLoss)) ** 2 - (BSHeight - UEHeight) ** 2)

def get_coverage_area_ratio(mcs, maxDist):
    if get_communication_distance(mcs) > maxDist and get_communication_distance(mcs+1) > maxDist: return 0
    elif get_communication_distance(mcs) > maxDist and get_communication_distance(mcs+1) <= maxDist: return (maxDist ** 2 - get_communication_distance(mcs+1) ** 2) / maxDist  ** 2
    else: return (get_communication_distance(mcs) ** 2 - get_communication_distance(mcs+1) ** 2) / maxDist  ** 2

In [ ]:
class node:
    def __init__(self, id = 0, minMCS = len(SNR)-1):
        self.id = id
        self.dist2parent = 0
        self.dist2child = 0
        self.buffer = {'access': list(), 'backhaul': list()}
        self.nextSlotType = None
        self.MCSDistribution = reversed([get_coverage_area_ratio(mcs, get_communication_distance(minMCS)) for mcs in range(1, len(SNR))])
        self.coverageDist = get_communication_distance(minMCS) * 2

    def generatePacket(self, slot):
        mcs = 1
        if random.random() < 1 / (trafficType['ftp3']['intensity'] ** -1 + 1):
            rand = random.random()
            while sum(self.MCSDistribution[:mcs]) < rand: mcs += 1
            packetSizeRBs = math.ceil([packetSize * 2 ** 20 / SpectralEfficiency[mcs] / 10 ** 6] / RBSizeMHz)
            self.buffer['access'].append(packet(packetSizeRBs, slot, self.id, "uplink"))
            return True
        else: return None

class packet:
    def __init__(self, size, slot, nodeId, direction):
        self.route = [nodeId]
        self.size = size
        self.initSlot = slot
        self.direction = direction
        


In [58]:
MCSDistribution = list(reversed([get_coverage_area_ratio(mcs, get_communication_distance(minMCS)) for mcs in range(1, len(SNR))]))
def generatePacket():
    mcs = 1
    rand = random.random()
    while sum(MCSDistribution[:mcs]) < rand: 
        mcs += 1
        print(rand, mcs)
    print(rand, mcs)
generatePacket()

0.37081357886029886 2
0.37081357886029886 3
0.37081357886029886 4
0.37081357886029886 4


23

In [106]:
1/31

0.03225806451612903